In [14]:
from matplotlib import animation
import numpy as np
import sympy as sp
import math
import matplotlib
import random
import matplotlib.pyplot as plt
from matplotlib import animation, rc, patches
from IPython.display import HTML
from matplotlib.patches import Polygon
L = 10.0
S = 5.0  # W/m
A0 = 0.1  # m^2
k = 2.0  # w/C/m
qn = 5.0  # 5 W/m^2


In [15]:
X_part = 2
Y_part = 2
Nx_nodes = X_part + 1
Ny_nodes = Y_part + 1
N_element = X_part * Y_part

# Generating nodes using meshgrid and reshaping
q = np.array([[0, 0], [L, 0], [0, L], [L, L]])  # 4 corners
NoN = (X_part + 1) * (Y_part + 1)  # number of nodes
NoE = X_part * X_part  # Number of  element
NPE = 4  # nodes per element

#x , y co-ordintaes
x, y = np.meshgrid(np.linspace(0, L, num=X_part + 1), np.linspace(0, L, num=Y_part + 1))

# Node list NL = [X co-ordinate, Y co-ordinate]
NL = np.column_stack([x.ravel(), y.ravel()])

# Elements
# EL = [ Node 1, node 2, node 3, node 4]
EL = np.zeros([NoE, NPE])  # element list
ELN = np.arange(1, NoE + 1)

for i in range(1, Y_part + 1):
    for j in range(1, X_part + 1):
        if j == 1:
            EL[(i - 1) * X_part + j - 1, 0] = (i - 1) * (X_part + 1) + j
            EL[(i - 1) * X_part + j - 1, 1] = EL[(i - 1) * X_part + j - 1, 0] + 1
            EL[(i - 1) * X_part + j - 1, 3] = EL[(i - 1) * X_part + j - 1, 0] + (X_part + 1)
            EL[(i - 1) * X_part + j - 1, 2] = EL[(i - 1) * X_part + j - 1, 3] + 1
        else:
            EL[(i - 1) * X_part + j - 1, 0] = EL[(i - 1) * X_part + j - 2, 1]
            EL[(i - 1) * X_part + j - 1, 3] = EL[(i - 1) * X_part + j - 2, 2]
            EL[(i - 1) * X_part + j - 1, 1] = EL[(i - 1) * X_part + j - 1, 0] + 1
            EL[(i - 1) * X_part + j - 1, 2] = EL[(i - 1) * X_part + j - 1, 3] + 1

# Number element with node = [element number, Node 1, node 2, node 3, node 4]
Num_EL = np.column_stack([ELN, EL[:, 0], EL[:, 1], EL[:, 2], EL[:, 3]])

element_nodes_coordinates = np.zeros((NoE, NPE, 2))  # Array to store coordinates for each element's nodes
# Each row corresponds to an element, and each element contains NPE nodes, each node with 2 coordinates (x, y).
# For example, EL = [Node 1, Node 2, Node 3, Node 4], where Node 1, Node 2, Node 3, and Node 4 are the coordinates in (x, y) format.
for i in range(NoE):
    for j in range(NPE):
        node_index = int(EL[i][j]) - 1  # Adjust for 0-based indexing
        element_nodes_coordinates[i][j] = NL[node_index]

element_nodes_coordinates


array([[[ 0.,  0.],
        [ 5.,  0.],
        [ 5.,  5.],
        [ 0.,  5.]],

       [[ 5.,  0.],
        [10.,  0.],
        [10.,  5.],
        [ 5.,  5.]],

       [[ 0.,  5.],
        [ 5.,  5.],
        [ 5., 10.],
        [ 0., 10.]],

       [[ 5.,  5.],
        [10.,  5.],
        [10., 10.],
        [ 5., 10.]]])

In [16]:

Gauss_2 = np.array([[1/np.sqrt(3.),-1/np.sqrt(3.), 1.], [1/np.sqrt(3.),1/np.sqrt(3.), 1.],[-1/np.sqrt(3.),1/np.sqrt(3.), 1.], [-1/np.sqrt(3.),-1/np.sqrt(3.), 1.]])

def N_func(z,n):
    N1 = (1 - z) * (1 - n) / 4
    N2 = (1 + z) * (1 - n) / 4
    N3 = (1 + z) * (1 + n) / 4
    N4 = (1 - z) * (1 + n) / 4

    return [N1,N2,N3,N4]

def dNzn_func(z,n):
    dN1z = -(1 - n) / 4
    dN1n = -(1 - z) / 4

    dN2z = (1 - n) / 4
    dN2n = -(1 + z) / 4

    dN3z = (1 + n) / 4
    dN3n = (1 + z) / 4

    dN4z = -(1 + n) / 4
    dN4n = (1 - z) / 4

    return [dN1z,dN1n],[dN2z,dN2n],[dN3z,dN3n],[dN4z,dN4n]


In [17]:

T_sols = []  # array to store all solutions
# Initial condition, make all nodes temperature 0
T_0 = np.zeros((NoN))
T_sols.append(T_0)

K = np.zeros((NoN, NoN))
M = np.zeros((NoN, NoN))
F_s = np.zeros(NoN)
F_q = np.zeros(NoN)
F_0 = np.zeros(NoN)

delta_t = 0.1
total_steps = 2000
for ts in range(total_steps):
    for el in range (NoE):
        global_indices = Num_EL[el][1:] - 1
        #created element coordinates list so that I can get coordinates for them and easy to run code for that nodes
        [[x1,y1],[x2,y2],[x3,y3],[x4,y4]]=element_nodes_coordinates[el]
        element_cord=np.array([[x1,y1],[x2,y2],[x3,y3],[x4,y4]])
        

        Ke=np.zeros((4,4))
        Me=np.zeros((4,4))
        Fes = np.zeros((4))
        Fe0 = np.zeros((4))
        
        #intial temperature
        T_indices = Num_EL[el][1:] - 1  # Correcting the indices to 0-based
        T1e = T_0[int(T_indices[0])]
        T2e = T_0[int(T_indices[1])]
        T3e = T_0[int(T_indices[2])]
        T4e = T_0[int(T_indices[3])]
        
        for ip in range(NPE):
            z=Gauss_2[ip][0]
            n=Gauss_2[ip][1]

            Grad_N=np.array([[dNzn_func(z,n)[0][0],dNzn_func(z,n)[1][0],dNzn_func(z,n)[2][0],dNzn_func(z,n)[3][0]],[dNzn_func(z,n)[0][1],dNzn_func(z,n)[1][1],dNzn_func(z,n)[2][1],dNzn_func(z,n)[3][1]]])
            
            J=np.dot(Grad_N,element_cord)
            det_J=np.linalg.det(J) 
            Nvec=np.array([N_func(z,n)[0],N_func(z,n)[1],N_func(z,n)[2],N_func(z,n)[3]])
            J_inv=np.linalg.inv(J)

            # K Matrix= Btranspse*D*B*det_J*wt
            B=np.dot(J_inv,Grad_N)

            D=np.array([[k,0],[0,k]])

            Ke+=np.dot(np.transpose(B),np.dot(D,B))*det_J*Gauss_2[ip, 2]
            # Mass Matrix

            Me+=det_J*Gauss_2[ip, 2]*np.outer(Nvec, Nvec)  

            # Body force for this element

            Fes += det_J*Gauss_2[ip, 2]*Nvec*S  # source term

            # INITIAL 'FORCE' VECTOR
            
            T_i = N_func(z,n)[0]*T1e + N_func(z,n)[1]*T2e+ N_func(z,n)[2]*T3e+ N_func(z,n)[3]*T4e
            Fe0 += det_J*Gauss_2[ip, 2]*Nvec*T_i
        

        # Assemble the elemental stiffness matrix into the global stiffness matrix
        global_indices = Num_EL[el][1:] - 1  # Correcting the indices to 0-based
        for i in range(NPE):
            global_i = int(global_indices[i])
            for j in range(NPE):
                global_j = int(global_indices[j])
                K[global_i, global_j] += Ke[i, j]  # Assemble Ke into K
                M[global_i, global_j] += Me[i, j]  # Assemble Me into M
        for i in range(NPE):
            global_i = int(global_indices[i])
            F_s[global_i] += Fes[i]  # Assemble Fes into F_s
            F_0[global_i] += Fe0[i]  # Assemble Fe0 into F_0
            